In [29]:
# parameters_content = '''
# #include <stdio.h>
# #include "include/gemmini.h"

# #define LEN(arr) ((int) (sizeof (arr) / sizeof (arr[0])))

# '''
# batch_size=32
# input_dim=3036
# param_file_name="parameters8.h"
# test_file_name="test8.c"
# if(batch_size%SYS_DIM !=0):
#     print('please use a batch size of SYS_DIM')
#     exit(1)   
# #layers = [input_dim,2500,2000,1500,1000,500,10]
# layers=[input_dim,4554,3036]
# mlp='// batch size: ' + str(batch_size) +'\n'
# mlp+='// before zeropad: '
# for l in layers:
#     mlp=mlp+str(l)+'x'
    
# parameters_content = parameters_content+mlp[:-1]+'\n'
# for i,l in enumerate(list(layers)):
#     layers[i] = int((l+SYS_DIM-1)/SYS_DIM)*SYS_DIM
    
# mlp='// after zeropad: '
# for l in layers:
#     mlp=mlp+str(l)+'x'
# parameters_content = parameters_content+mlp[:-1]+'\n'

# input_mat = 'static elem_t input_mat[{0}][{1}] row_align(1)= '.format(batch_size,layers[0]) + '{0};\n'
# parameters_content = parameters_content+input_mat;

# matmuls = []
# for i,(l1,l2) in enumerate(zip(layers[:-1],layers[1:])):
#     matrix = 'static elem_t weights'+str(i)+'['+str(l1)+']'+'['+str(l2)+'] row_align(1)= {0};\n'
#     parameters_content = parameters_content+matrix
#     inter_results = 'static elem_t inter_results'+str(i)+'['+str(batch_size)+']'+'['+str(l2)+'] row_align(1)= {0};\n'
#     parameters_content = parameters_content+inter_results
#     if i==0:
#         matmuls.append(('input_mat',batch_size,'weights'+str(i),l1,'inter_results'+str(i),l2))
#     else:
#         matmuls.append(('inter_results'+str(i-1),batch_size,'weights'+str(i),l1,'inter_results'+str(i),l2))

# f = open(param_file_name, "w")
# f.write(parameters_content)
# f.close()
from math import sqrt
test_file_name='resnet50-16.c'
SYS_DIM=16
test_content= '''

#include <stdio.h>
#include <string.h>
#include <stdbool.h>
#include <sys/mman.h>
#include "include/gemmini.h"

#define verbose(layer_num,old_C,filter,C) printf("layer %d: operand %d %d filter %d %d result %d %d\\n", layer_num, LEN(old_C),LEN(old_C[0]),LEN(filter),LEN(filter[0]),LEN(C),LEN(C[0]));
#define LEN(arr) ((int) (sizeof (arr) / sizeof (arr[0])))
#define N 224


static void tensor_reshape(int channels, elem_t in_tensor[][channels],int dim1,int dim2,int kdim, int stride, elem_t out_tensor[][channels]){
    int w1,w2,channel,i,j;
    int k = 0;
    int row = 0;
    for (w1=0;w1<dim1;w1+=stride){
        for(w2=0;w2<dim2;w2+=stride){
            k=0;
            for(channel = 0; channel<channels; channel++){
                for(i=-kdim/2;i<=kdim/2;i++){
                    for(j=-kdim/2;j<=kdim/2;j++){
                        if(i+w1<0 || i+w1>=dim1||j+w2<0 ||j+w2>=dim2)
                            out_tensor[row][k]=0;
                        else
                            out_tensor[row][k]=in_tensor[dim2*(i+w1)+j+w2][channel];
                        k++;
                    }
                }
            }
            row++; 
        }
    }
}    
static void avg_pool7(int len, elem_t in[][len],elem_t out[][len]){
    int i, j;
    for(i=0;i<len;i++){
        for(j=0;j<7*7;j++){
            out[0][i] += in[j][i];
        }
        out[0][i]=out[0][i]/49;
    }
}

static void rocket_fix_strided_dimension(int img_dim, int len2, elem_t in[][len2], int len3, elem_t out[][len3]){
        
    for(int i =0;i<img_dim;i+=2){
        for(int k = 0; k<img_dim;k+=2)
            for(int j=0;j<len2; j+=1){
                out[(i/2)*img_dim+k/2][j]=in[i*img_dim+k][j];

            }
    }
}

static void rocket_zeropad(int dim1,int dim2, elem_t in[][dim2], int dim3, elem_t out[][dim3]){
    for(int i = 0; i<dim1;i++){
        for(int j =0; j<dim2;j++){
            out[i][j] = in[i][j];
        }
    }
}

static void tiled_matmul_compare(size_t DIM_I, size_t DIM_J, size_t DIM_K,
        // elem_t A[DIM_I][DIM_K], elem_t B[DIM_K][DIM_J], acc_t D[DIM_I][DIM_J],
        elem_t A[DIM_I][DIM_K], elem_t B[DIM_K][DIM_J], void * D,
        elem_t C[DIM_I][DIM_J],
        int act, int shift, int relu6_shift, int full_bias_width,
        enum tiled_matmul_type_t tiled_matmul_type,
        bool compare, char * layer_name)
{
    if (compare)
        printf("%s: gemmini\\n", layer_name);
    tiled_matmul_option(DIM_I, DIM_J, DIM_K,
        A, B, D, C, act, shift, relu6_shift, full_bias_width,
        tiled_matmul_type);

    if (compare) {
        printf("%s: CPU\\n", layer_name);
        elem_t gold[DIM_I][DIM_J];
        tiled_matmul_option(DIM_I, DIM_J, DIM_K,
            A, B, D, gold, act, shift, relu6_shift, full_bias_width,
            CPU);

        if (!MAT_IS_EQUAL(DIM_I, DIM_J, C, gold)) {
            printf("Layer calculated incorrectly: %s\\n", layer_name);
            exit(1);
        }
    }
}   

int main (int argc, char * argv[]) {
#ifndef BAREMETAL
    if (mlockall(MCL_CURRENT | MCL_FUTURE) != 0) {
      perror("mlockall failed");
      exit(1);
    }
#endif

    matmul_flush(0);

    enum tiled_matmul_type_t tiled_matmul_type;
    if (argc < 2) {
        // printf("usage: %s matmul_option\\n  matmul_option may be 'os', 'ws', or cpu'\\n");
        // exit(0);
        tiled_matmul_type = OS;
    } else if (strcmp(argv[1], "cpu") == 0) {
        tiled_matmul_type = CPU;
    } else if (strcmp(argv[1], "os") == 0) {
        tiled_matmul_type = OS;
    } else if (strcmp(argv[1], "ws") == 0) {
        tiled_matmul_type = WS;
    }

    bool compare;
    if (argc < 3) {
        compare = false;
    } else if (strcmp(argv[2], "compare") == 0) {
        compare = true;
    } else {
        printf("Unknown command-line argument\\n");
        exit(1);
    }


'''
model = [(7,3,64,2),(3,64,64,2)]+[(1,64,64,1),(3,64,64,1),(1,64,256,1)]+[(1,64,64,1),(3,64,64,1),(1,64,256,1)]*2+[(1,256,128,1),(3,128,128,2),(1,128,512,1)]+[(1,256,128,1),(3,128,128,1),(1,128,512,1)]*3+[(1,512,256,1),(3,256,256,2),(1,256,1024,1)]+[(1,512,256,1),(3,256,256,1),(1,256,1024,1)]*5+[(1,1024,512,1),(3,512,512,2),(1,512,2048,1)]+[(1,1024,512,1),(3,512,512,1),(1,512,2048,1)]*2

test_content += '    static elem_t img[N*N][3] = {0};\n'
test_content =  test_content + '    unsigned long cycles['+str(len(model)+2)+']={0};\n'
test_content =  test_content + '    unsigned long start,end;\n'
test_content =  test_content + '    start = read_cycles();\n'

def zeropadded_value(dim):
    return int((dim+SYS_DIM-1)/SYS_DIM)*SYS_DIM

def conv_to_matrix(filter_dim,img_dim,input_channels,output_channels,stride,test_content,layer,prev_tensor_name):
    kernel_rows = zeropadded_value(filter_dim*filter_dim*input_channels)
    kernel_cols = zeropadded_value(output_channels)
    tensor_rows = zeropadded_value(int((img_dim+stride-1)/stride)*int((img_dim+stride-1)/stride))
    tensor_cols = kernel_rows
    test_content += '    static elem_t kernel'+str(layer)+'['+str(kernel_rows)+']'+'['+str(kernel_cols)+'] row_align(1)= {0};\n'
    test_content += '    static elem_t tensor'+str(layer)+'['+str(tensor_rows)+']'+'['+str(tensor_cols)+'] row_align(1)= {0};\n' 
    test_content += '    static elem_t result'+str(layer)+'['+str(tensor_rows)+']'+'['+str(kernel_cols)+'] row_align(1)= {0};\n' 

    if(filter_dim>1):
        test_content += '    tensor_reshape('+str(input_channels)+',' + prev_tensor_name+','+str(img_dim)+', '+str(img_dim)+', '+ str(filter_dim)+', '+str(stride)+', tensor'+str(layer)+');\n'
    return test_content,tensor_rows,kernel_cols,kernel_rows
        
def matmul_layer(test_content,layer,tensor_rows,kernel_cols,kernel_rows,new_result_name,fix_dimensions = False):
    
    test_content +='\n\n    /* matmul number: {0} */\n\n'.format(str(layer))
    test_content +='    tiled_matmul_compare({0}, {1}, {2},    // dimensions\n'.format(tensor_rows,kernel_cols,kernel_rows)
    if not fix_dimensions:
        test_content +='    {0}, {1}, NULL, {2},      // addresses\n'.format('tensor'+str(layer),'kernel'+str(layer),new_result_name)
    else:
        test_content +='    {0}, {1}, {2}, {3},      // addresses\n'.format('tensor'+str(layer),'kernel'+str(layer),'inter_result'+str(layer),new_result_name)

    test_content +='    RELU, 0, 0, 0,              // activation, shift, r6_shift, full_width_bias\n'
    test_content +='    tiled_matmul_type, compare, "layer_'+str(layer)+'");\n'
    test_content +='    // verbose({0},{1},{2},{3})\n'.format(layer,'tensor'+str(layer),'kernel'+str(layer),new_result_name)
    test_content +='    /* end of matmul number: {0} */\n\n'.format(str(layer))
    test_content +='    end = read_cycles();\n'
    test_content +='    cycles['+str(layer)+'] = end-start;\n'
    test_content +='    start = end;\n'
    return test_content

#(kernel_dim,input_channels,output_channels,stride)
img_dim = 224
prev_layers = []
for m,(kernel_dim,input_channels,output_channels,stride) in enumerate(model):
    prev_res_name='result'+str(m-1)
    new_result_name = 'result'+str(m)
    if m == 0:
        prev_res_name = 'img'
    test_content,tensor_rows,kernel_cols,kernel_rows = conv_to_matrix(kernel_dim,img_dim,input_channels,
                                                                      output_channels,stride,test_content,
                                                                      m,prev_res_name)
    prev_layers.append((tensor_rows,kernel_cols,kernel_rows))
    fix_dimensions = False
    if m >=4:
        if (m-4)%3 == 0:
            
            tr,kc,kr = prev_layers[m-2]
            if tr != tensor_rows:
                test_content += '    static elem_t inter_result'+str(m)+'['+str(tensor_rows)+']'+'['+str(kernel_cols)+'] row_align(1)= {0};\n' 
                test_content += '    rocket_fix_strided_dimension('+str(int(sqrt(tr)))+','+str(kc)+',result' + str(m-2)+',' +str(kernel_cols)+',  inter_result'+str(m)+');\n'
            elif kc != kernel_cols and tr==tensor_rows:
                test_content += '    static elem_t inter_result'+str(m)+'['+str(tensor_rows)+']'+'['+str(kernel_cols)+'] row_align(1)= {0};\n' 
                test_content += '    rocket_zeropad('+str(tr)+','+str(kc)+',result' + str(m-2)+',' +str(kernel_cols)+',  inter_result'+str(m)+');\n'
            
            fix_dimensions = True
            
    test_content = matmul_layer(test_content,m,tensor_rows,kernel_cols,kernel_rows,new_result_name,fix_dimensions)
    img_dim=int(img_dim/stride)


test_content +='\n\n    /* AVG Pool: {0} */\n\n'.format(str(len(model)))
test_content += '    static elem_t tensor'+str(len(model)+1)+'['+str(SYS_DIM)+']'+'['+str(kernel_cols)+'] row_align(1)= {0};\n' 
test_content +='    avg_pool7(2048,'+'result'+str(len(model)-1)+',tensor'+str(len(model)+1)+');\n'
test_content +='    end = read_cycles();\n'
test_content +='    cycles['+str(len(model))+'] = end-start;\n'
test_content +='    start = end;\n'


test_content += '\n\n    static elem_t kernel'+str(len(model)+1)+'['+str(2048)+']'+'['+str(zeropadded_value(1000))+'] row_align(1)= {0};\n'
test_content += '    static elem_t result'+str(len(model)+1)+'['+str(SYS_DIM)+']'+'['+str(zeropadded_value(1000))+'] row_align(1)= {0};\n' 

test_content = matmul_layer(test_content,len(model)+1,SYS_DIM,zeropadded_value(1000),2048,'result'+str(len(model)+1))


test_content +='''
    unsigned long overall_cycles = 0;
    for(int cyc = 0; cyc < '''+str(len(model)+2)+''' ; cyc++){
        overall_cycles += cycles[cyc];
    }
    for(int cyc = 0; cyc < '''+str(len(model)+2)+''' ; cyc++){
        printf("Cycles taken in layer %d: %lu, %lf\\n", cyc,cycles[cyc],cycles[cyc]*1.0/(1.0*overall_cycles));
    }
    printf("Overall cycles taken: %lu\\n",overall_cycles);
\n
    return 0;
}\n
''' 

f = open(test_file_name, "w")
f.write(test_content)
f.close()